In [2]:
import pandas as pd

In [7]:
africa_iso_code = pd.read_csv('./africa_iso_codes.csv')
hs4_2digit = pd.read_csv('./products_prospedia.csv', dtype={'code': str})
hs4_2digit = hs4_2digit[hs4_2digit['level'] == '2digit'][['name_short_en','code', 'parent_id']]
china_iso_code = 'CHN'

## Cleaning because some dumbfuck decided that codes should be numbers and not strings,
## And some codes that had 0XXX became XXX.
## Good fucking Game

def mutate_code_to_4_digit(value):
    if len(value) > 4:
        return value
    return (4 - len(value)) *'0' + value
        

In [8]:
years = [2009, 2010, 2011, 2012, 2013, 2014]

In [10]:
for year in years:
    trade_data = pd.read_csv('./HS92_ccp_%s.csv'%year, dtype={'commoditycode': str} )
    africa_trade_data = trade_data.loc[
        (trade_data['importer'] == china_iso_code) & \
        (trade_data['exporter'].isin(africa_iso_code['3digit']))
    ]
    
    africa_trade_data = africa_trade_data.assign(commoditycode = lambda x: x['commoditycode'].map(mutate_code_to_4_digit))
    africa_trade_data = africa_trade_data.assign(commodity_group = lambda x: x['commoditycode'].map(lambda y: y[0:2]))

    data = africa_trade_data.groupby(['commodity_group', 'exporter', 'year']).sum()
    data = data.reset_index()
    data = data.merge(africa_iso_code, left_on='exporter', right_on='3digit', how='left')
    data = data.merge(hs4_2digit, left_on='commodity_group', right_on='code')
    data.rename(columns={'name': 'country_name', 'name_short_en': 'product_name'}, inplace=True)
    data = data[['commodity_group','product_name', 'export_value', 'import_value','country_name', '2digit', '3digit','year']]

    data.to_json('./visualization_data_%s.json' %year, orient='records')